In [ ]:
# ---------------------------------------
# HW9 VERIFICATION CODE (Q1–Q4)
# ---------------------------------------

import onnxruntime as ort
import numpy as np
from io import BytesIO
from urllib import request
from PIL import Image


def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    return Image.open(stream)

def prepare_image(img, target_size=(128, 128)):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

def preprocess(img):
    x = np.array(img).astype("float32")
    x = x / 255.0      # scale
    x = (x - 0.5) * 2  # normalization from HW8
    # first pixel R-value:
    print("First pixel R-channel:", x[0, 0, 0])
    x = np.transpose(x, (2, 0, 1))  # CHW format
    x = np.expand_dims(x, axis=0)
    return x


url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
img = download_image(url)
img = prepare_image(img)


x = preprocess(img)


model_path = "hair_classifier_v1.onnx"
session = ort.InferenceSession(model_path)

# show input and output names (Q1)
print("Input names:", [i.name for i in session.get_inputs()])
print("Output names:", [o.name for o in session.get_outputs()])


output_name = session.get_outputs()[0].name
pred = session.run([output_name], {"input": x})[0]

print("MODEL OUTPUT:", float(pred[0][0]))
